# Project 3: Understanding User Behavior

## 1. player_status.db set up: create_databsae.ipynb

## 2. Docker Set up

In [ ]:
!docker-compoes up -d

In [ ]:
!docker-compose ps

In [ ]:
!docker-compose exec kafka \
  kafka-topics \
    --create \
    --topic events \
    --partitions 1 \
    --replication-factor 1 \
    --if-not-exists \
    --zookeeper zookeeper:32181

## 3. API 

In [ ]:
!docker-compose exec mids \
env FLASK_APP=/w205/project-3-haoyuzhang89/game_api.py \
flask run --host 0.0.0.0

## 4. Purchase a sword/shield

In [ ]:
!docker-compose exec mids \
ab \
-n 10 \
-H "Host: user1.comcast.com" \
http://localhost:5000/purchase_a_sword/Haoyu

In [ ]:
!docker-compose exec mids \
ab \
-n 10 \
-H "Host: user1.comcast.com" \
http://localhost:5000/purchase_a_shield/Hao

## Building Realtime Data

In [ ]:
!while true; do
docker-compose exec mids \
ab -n 10 -H "Host: user1.comcast.com" \
http://localhost:5000/purchase_a_sword/Haoyu 
sleep 10
done

In [ ]:
!while true; do
docker-compose exec mids \
ab -n 10 -H "Host: user2.att.com" \
http://localhost:5000/purchase_a_shield/Haolun
sleep 10
done

In [ ]:
!while true; do
docker-compose exec mids \
ab -n 10 -H "Host: user3.huawei.com" \
http://localhost:5000/purchase_a_sword/Hao
sleep 10
done

## Realtime Writing

### Sword_purchases

In [ ]:
!docker-compose exec spark \
spark-submit \
/w205/project-3-haoyuzhang89/write_swords_stream.py

In [ ]:
## Check in another terminal 
!docker-compose exec cloudera hadoop fs -ls /tmp/sword_purchases

### Hive & Presto

In [ ]:
!docker-compose exec cloudera hive

In [ ]:
create external table if not exists default.sword_purchases (
    raw_event string,
    timestamp string,
    Accept string,
    Host string,
    User_Agent string,
    event_type string,
    rarity string
  )
  stored as parquet
  location '/tmp/sword_purchases'
  tblproperties ("parquet.compress"="SNAPPY");

hive> create external table if not exists default.sword_purchases (
    >     raw_event string,
    >     timestamp string,
    >     Accept string,
    >     Host string,
    >     User_Agent string,
    >     event_type string,
    >     rarity string
    >   )
    >   stored as parquet
    >   location '/tmp/sword_purchases'
    >   tblproperties ("parquet.compress"="SNAPPY");
OK
Time taken: 5.794 seconds

In [ ]:
!docker-compose exec presto presto --server presto:8080 --catalog hive --schema default

In [ ]:
presto:default> select count(*) from sword_purchases;

In [ ]:
presto:default> select count(*), rarity  from sword_purchases group by rarity;

In [ ]:
presto:default> select count(*) as number, rarity  from shield_purchases group by rarity order by number desc;

In [ ]:
presto:default> select * from sword_purchases limit 10;

### Shield_purchases

In [ ]:
!docker-compose exec spark \
spark-submit \
/w205/project-3-haoyuzhang89/write_shields_stream.py

In [ ]:
## Check in another terminal 
!docker-compose exec cloudera hadoop fs -ls /tmp/shield_purchases

### Hive & Presto

In [ ]:
!docker-compose exec cloudera hive

In [ ]:
create external table if not exists default.shield_purchases (
    raw_event string,
    timestamp string,
    Accept string,
    Host string,
    User_Agent string,
    event_type string,
    rarity string
  )
  stored as parquet
  location '/tmp/shield_purchases'
  tblproperties ("parquet.compress"="SNAPPY");

In [ ]:
!docker-compose exec presto presto --server presto:8080 --catalog hive --schema default

In [ ]:
presto:default> select count(*) from shield_purchases;

In [ ]:
presto:default> select count(*) as number, rarity from shield_purchases group by rarity order by number desc;

In [ ]:
select * from shield_purchases limit 10;